[Weekopdracht 6 2022](https://community-challenge.netlify.app/) - &copy; Paul Schouten 2022

# Is it in Stock?

Al vele jaren gaat Klaas naar de supermarkt bij hem om de hoek. Hij doet er niet alleen boodschappen voor zichzelf, maar ook voor zijn mindervalide buurvrouw. Elke dinsdag krijgt hij een boodschappenlijstje mee en brengt hij de boodschappen weer netjes bij haar thuis af.
Klaas heeft echter recent een extra baan gekregen en wil dit proces optimaliseren om zo meer tijd te besparen. Aan de hand van de volgende opdrachten ga jij hem helpen zo efficiënt mogelijk de boodschappen te doen.

<div style="font-size: 2em; text-align: center;">★</div>

Adrie heeft het boodschappenlijstje voor hem klaargelegd. Zijn alle producten op het lijstje ook te vinden in de supermarkt? Het assortiment is [hier](assortiment.json) te vinden.

Uitwerking: Eerst worden de JSON files ingelezen. Deze worden omgezet naar een list-of-named-tuples. Dit is (wellicht) handiger voor de vervolgstappen. Helaas blijkt dat de items op het boodschappenlijstje niet helemaal overeenkomen met de items in het assortiment. Om toch de producten te kunnen vinden in de winkel, wordt gebruikt gemaakt van een _fuzzy search_. Bij deze eerste opdracht wordt alleen gekeken of een product in het assortiment te vinden is.


In [1]:
import json
from thefuzz import process
import collections


# Inlezen json files
with open('lijstje.json') as f:
    boodschappenlijst = json.load(f)["lijstje"]
    
with open('assortiment.json') as f:
    assortiment = json.load(f)["producten"]

    
# boodschap is een _namedtuple_, voorbereiden op komende opdrachten    
boodschap = collections.namedtuple(
    'boodschap', 
    ['product', 'hoeveelheid', 'eenheid', 'aantal', 'subtotaal'],
    defaults = (None, None, None, None, 999))
assorti = collections.namedtuple(
    'assorti', 
    ['product', 'schap', 'gewicht', 'eenheid', 'prijs'])

# Omzetten list-van-dicts naar list-van-namedtuple
lijstje = [ boodschap(**item) for item in boodschappenlijst ]
assortiment = [ assorti(**item) for item in assortiment ]

# Lijst met producten
producten = [ product.product for product in assortiment ]

# Loop over de boodschappenlijst
# Helaas geen exacte match tussen boodschappenlijst en assortiment,
#     --> thefuzz voor ongeveer matchen met strings
# Iedere item van het lijstje wordt opgezocht in het assortiment,
#     bij een match wordt het product toegevoegd aan gevonden_boodschappen
gevonden_boodschappen = []
for item in lijstje:
    print(f'Zoeken naar {item.product}: ')
    
    # Fuzzy string search, geeft hoogste score terug als 
    # tuple (string, score)
    gevonden = process.extract(item.product, producten)
    
    for vondst, score in gevonden:
        if score >= 90:
            # Goede match met gezocht item
            print(f'    Gevonden keuze: {vondst}')
            gevonden_boodschappen.append(item.product)

# Als alle boodschappen gevonden zijn, moet de set van gevonden_boodschappen
# even lang zijn als het (boodschappen)lijstje
print('')
if len(lijstje) == len(set(gevonden_boodschappen)):
    print('Alle boodschappen zijn gevonden!')
else:
    print('Niet alle boodschappen zijn gevonden...')


Zoeken naar melk: 
    Gevonden keuze: melk (literpak)
    Gevonden keuze: melk (groot)
    Gevonden keuze: karnemelk
Zoeken naar gildekorn: 
    Gevonden keuze: gildekorn
Zoeken naar cassis: 
    Gevonden keuze: cassis
Zoeken naar milde kwark: 
    Gevonden keuze: milde kwark
Zoeken naar mars: 
    Gevonden keuze: mars
Zoeken naar paprika chips: 
    Gevonden keuze: chips (paprika)
Zoeken naar eieren: 
    Gevonden keuze: eieren (6-pak)
    Gevonden keuze: eieren (dozijn)

Alle boodschappen zijn gevonden!


<div style="font-size: 2em; text-align: center;">★★</div>

Klaas krijgt altijd 15 euro mee om de boodschappen mee te doen. Heeft Klaas hier genoeg aan? De prijzen zijn te vinden in het [assortiment](assortiment.json).

Uitwerking: Het boodschappenlijstje wordt opnieuw bekeken, voor ieder product wat erop staat wordt opgezocht wat de prijs per stuk is. Het benodige aantal wordt ook uitgerekend, dit is de gewenste _hoeveelheid_ gedeeld door het _gewicht_. Het aantal wordt naar boven afgrond. Immers, als je 8 eieren wilt hebben, dan heb je er niet genoeg aan 6...


In [10]:
import math

# Eerst alle booschappen vinden in het assortiment,
# nu ook het benodigde aantal en het subtotaal uitrekenen
gevonden_boodschappen = []
for item in lijstje:
    print(f'Zoeken naar {item.product}, hoeveelheid: {item.hoeveelheid} {item.eenheid} ')
    
    # Fuzzy string search, geeft scores terug als 
    # tuple (string, score)
    gevonden = process.extract(item.product, producten)
    
    # Loop over ieder gevonden item in het assortiment
    beste_keuze = boodschap()
    
    for vondst, score in gevonden:
        # Bij een score van 90 worden goede producten gevonden. 
        # Gelukkig is de karnemelk duur...
        if score >= 90:
            # Goede match met gezocht product
            print(f'    Gevonden: {vondst} ', end='')
            assorti_keuze = [ product for product in assortiment if product.product == vondst ]
            
            # Uitzoeken hoeveel uit het assortiment te kopen,
            # afronden naar boven
            # subtotaal uitrekenen
            aantal_nodig = math.ceil(item.hoeveelheid / assorti_keuze[0].gewicht)
            subtotaal = aantal_nodig * assorti_keuze[0].prijs
            print(f'{aantal_nodig} x {assorti_keuze[0].prijs} = {subtotaal}', end=' ')
            
            if subtotaal < beste_keuze.subtotaal:
                beste_keuze = boodschap(
                                    assorti_keuze[0].product, 
                                    item.hoeveelheid,
                                    item.eenheid,
                                    aantal_nodig,
                                    subtotaal)
                # Er is een nieuwe beste keus
                print('*')
            else:
                # Naar de volgende regel
                print('')
    
    # De beste keuze voor item uit lijstje toevoegen aan de gevonden boodschappen
    
    gevonden_boodschappen.append(beste_keuze)
    print()

# Nu het totaalbedrag uitrekenen
totaal_prijs = [ item.subtotaal for item in gevonden_boodschappen ]
totaal = sum(totaal_prijs)

print(f'Klaas moet €{totaal:.2f} betalen aan de kassa.', end=' ')
if totaal <= 15:
    print('Dit is minder dan €15, Klaas komt uit met het bedrag')
else:
    print(f'Klaas zal €{totaal-15:.2f} moeten bijbetalen...')

Zoeken naar melk, hoeveelheid: 2.4 liter 
    Gevonden: melk (literpak) 3 x 1.19 = 3.57 *
    Gevonden: melk (groot) 1 x 2.24 = 2.24 *
    Gevonden: karnemelk 3 x 1.32 = 3.96 

Zoeken naar gildekorn, hoeveelheid: 2 stuks 
    Gevonden: gildekorn 2 x 1.87 = 3.74 *

Zoeken naar cassis, hoeveelheid: 1 liter 
    Gevonden: cassis 1 x 1.37 = 1.37 *

Zoeken naar milde kwark, hoeveelheid: 500 gram 
    Gevonden: milde kwark 1 x 1.19 = 1.19 *

Zoeken naar mars, hoeveelheid: 1 multipack 
    Gevonden: mars 1 x 2.9 = 2.9 *

Zoeken naar paprika chips, hoeveelheid: 1 zak 
    Gevonden: chips (paprika) 1 x 1.14 = 1.14 *

Zoeken naar eieren, hoeveelheid: 8 stuks 
    Gevonden: eieren (6-pak) 2 x 2.14 = 4.28 *
    Gevonden: eieren (dozijn) 1 x 3.68 = 3.68 *

Klaas moet €16.26 betalen aan de kassa. Klaas zal €1.26 moeten bijbetalen...


<div style="font-size: 2em; text-align: center;">★★★</div>

De buurvrouw is jarig en wilt graag een taart bakken. Aangezien ze niet zo van het weggooien is wilt ze altijd goed kijken naar de hoeveelheid ingredienten en niet teveel kopen. Hoeveel blijft er over van de ingrediënten voor het [taartrecept](recept.json), als de boodschappen zo gedaan worden met de minste verspilling?



## Bonus

Binnenin de supermarkt is de plek van de producten bekend. Met behulp van [Dijkstra's algoritme](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) kan je kijken wat de kortste route in de supermarkt is. Zo kan je de looproute optimaliseren en dus weer sneller thuis zijn.

Hieronder is een schematische weergave van de supermarkt toegevoegd. Bereken de korste route aan de hand van het boodschappenlijstje van de eerste opdracht. Uiteraard moet je langs elk schap van het product op je lijstje.

<img src="supermarkt.png" style="width: 50%; display: block; margin-left: auto; margin-right: auto;" />